In [2]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

In [3]:
#Cargamos los CSV y comprobamos
df1 = pd.read_csv("historical_data.csv")
df2 = pd.read_csv("historical_data_day2.csv")
df1.head()

,Date,Symbol,Open,High,Low,Close,Volume
0,2024-10-28,AAPL,233.320,234.730,232.55,233.40,36087134
1,2024-10-25,AAPL,229.740,233.220,229.57,231.41,38802304
2,2024-10-24,AAPL,229.980,230.820,228.41,230.57,31109503
3,2024-10-23,AAPL,234.080,235.144,227.76,230.76,52286979
4,2024-10-22,AAPL,233.885,236.220,232.60,235.86,38846578


In [4]:
df2.head()

,Date,Symbol,Open,High,Low,Close,Volume
0,2024-10-29,COST,890.00,891.1700,874.09,886.99,1361681
1,2024-10-28,COST,895.99,895.9900,890.00,890.96,912507
2,2024-10-25,COST,895.65,901.6191,890.00,891.22,1425842
3,2024-10-24,COST,901.50,903.5000,888.26,893.42,1649565
4,2024-10-23,COST,896.02,905.6800,894.99,899.17,2193473


In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143301 entries, 0 to 143300
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Date    143301 non-null  object 
 1   Symbol  143301 non-null  object 
 2   Open    143301 non-null  float64
 3   High    143301 non-null  float64
 4   Low     143301 non-null  float64
 5   Close   143301 non-null  float64
 6   Volume  143301 non-null  int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 7.7+ MB


In [6]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146242 entries, 0 to 146241
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Date    146242 non-null  object 
 1   Symbol  146242 non-null  object 
 2   Open    146242 non-null  float64
 3   High    146242 non-null  float64
 4   Low     146242 non-null  float64
 5   Close   146242 non-null  float64
 6   Volume  146242 non-null  int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 7.8+ MB


In [7]:
df = pd.concat([df1, df2], ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289543 entries, 0 to 289542
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Date    289543 non-null  object 
 1   Symbol  289543 non-null  object 
 2   Open    289543 non-null  float64
 3   High    289543 non-null  float64
 4   Low     289543 non-null  float64
 5   Close   289543 non-null  float64
 6   Volume  289543 non-null  int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 15.5+ MB


In [23]:
#Pasamos la columna date a formato fecha
df['Date'] = pd.to_datetime(df['Date'])

#Ordenamos por ticker y fecha
df = df.sort_values(by=['Symbol', 'Date'])
df = df.reset_index(drop=True)

#Calculamos la variación porcentual diaria del valor de cierre de cada accion y lo guardamos en una nueva columna
df['Pct_Change'] = round(df.groupby('Symbol')['Close'].pct_change()*100, 2)

#Filtramos las filas donde la variación porcentual sea mayor al 10%
significant_changes_df = df[(df['Pct_Change'] > 10) | (df['Pct_Change'] < -10)].reset_index(drop=True)

#Mostramos el resultado
significant_changes_df

,Date,Symbol,Open,High,Low,Close,Volume,Pct_Change
0,2000-01-13,AAPL,94.48,98.75,92.50,96.75,9220400,10.96
1,2000-03-01,AAPL,118.56,132.06,118.50,130.31,9616100,13.69
2,2000-04-17,AAPL,109.50,123.94,109.06,123.87,3656800,10.73
3,2000-06-21,AAPL,50.50,56.94,50.31,55.63,4375000,-45.06
4,2000-09-29,AAPL,28.19,29.00,25.37,25.75,132529300,-51.87
...,...,...,...,...,...,...,...,...
1503,2020-03-12,XOM,39.90,40.65,37.00,37.18,65046051,-11.43
1504,2020-03-18,XOM,34.55,36.71,31.23,33.12,53791453,-10.02
1505,2020-03-24,XOM,34.42,36.17,33.33,35.44,61774908,12.69
1506,2020-04-17,XOM,39.88,43.32,39.87,43.22,40799138,10.40


In [16]:
#Top 10 compañias con más días en los que el precio de la accion se ha movido más de un 10%
significant_changes_df.groupby(["Symbol"])["Pct_Change"].count().sort_values(ascending=False).head(10)

Symbol
AMD     167
NVDA    124
AMZN    104
NFLX    101
TSLA     79
BAC      67
MS       66
ADBE     64
WFC      53
ORCL     52
Name: Pct_Change, dtype: int64

In [21]:
#Sacamos numeritos (evolución desde el inicio de la serie historica, media de variación diaria, max y min de vaiación en 1 día, días de la serie historica)
round(df.groupby(["Symbol"])["Pct_Change"].agg(["sum","mean","max", "min", "count"]), 3)

,sum,mean,max,min,count
Symbol,,,,,
AAPL,509.69,0.081,13.90,-85.49,6287
ABBV,209.50,0.070,13.77,-16.25,2975
ABT,196.96,0.031,12.47,-51.07,6288
ACN,419.32,0.072,16.37,-13.45,5856
ADBE,458.66,0.073,23.99,-50.07,6288
AMD,705.59,0.112,52.29,-49.49,6288
AMGN,274.37,0.044,15.10,-49.11,6288
AMZN,608.68,0.097,34.47,-94.90,6287
AVGO,474.57,0.124,15.83,-89.92,3832


In [31]:
#Buscamos el valor con mayor bajada en un día
df.sort_values("Pct_Change",ascending=True).head(1)

,Date,Symbol,Open,High,Low,Close,Volume,Pct_Change
59257,2010-01-21,BRK.B,71.13,73.43,70.51,72.72,289944,-97.91


In [30]:
#Buscamos el valor con mayor subida en un día
df.sort_values("Pct_Change",ascending=False).head(1)

,Date,Symbol,Open,High,Low,Close,Volume,Pct_Change
175436,2008-10-13,MS,15.43,19.06,12.66,18.1,199684000,86.98


In [32]:
#Vemos como reaccionó el mercado ante la presentación del iphone 16 (9/9/24)
df[(df["Symbol"] == "AAPL") & (df["Date"] <= "2024-09-10") & (df["Date"] >= "2024-09-06")]

,Date,Symbol,Open,High,Low,Close,Volume,Pct_Change
6251,2024-09-06,AAPL,223.95,225.24,219.77,220.82,48423011,-0.70
6252,2024-09-09,AAPL,220.82,221.27,216.71,220.91,67179965,0.04
6253,2024-09-10,AAPL,218.92,221.48,216.73,220.11,51591033,-0.36


In [43]:
#Hacemos un nuevo df con el historico de 2023 y otro solo con los días con variación de más del 10%
only_2023_df = df[(df["Date"] >= "2023-01-01") & (df["Date"] <= "2023-12-31")].reset_index(drop=True)
big_moves_2023_df = only_2023_df[(only_2023_df['Pct_Change'] > 10) | (only_2023_df['Pct_Change'] < -10)].reset_index(drop=True)


In [35]:
#Sacamos días de gran volatilidad por compañía
big_moves_2023_df.groupby(["Symbol"])["Pct_Change"].count().sort_values(ascending=False)

Symbol
TSLA     4
AMD      2
META     2
NVDA     2
ORCL     2
AVGO     1
CMCSA    1
CRM      1
DHR      1
LLY      1
NFLX     1
NKE      1
Name: Pct_Change, dtype: int64

In [37]:
#Sacamos el historico de Apple de 2023
apple_2023_df = df[(df["Date"] >= "2023-01-01") & (df["Date"] <= "2023-12-31") & (df["Symbol"] == "AAPL")].reset_index(drop=True)
apple_2023_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        250 non-null    datetime64[ns]
 1   Symbol      250 non-null    object        
 2   Open        250 non-null    float64       
 3   High        250 non-null    float64       
 4   Low         250 non-null    float64       
 5   Close       250 non-null    float64       
 6   Volume      250 non-null    int64         
 7   Pct_Change  250 non-null    float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 15.8+ KB


In [38]:
#Guardamos el df de Apple en un CSV
apple_2023_df.to_csv("Apple_2023.csv", index=False)

In [44]:
#En el df de todo 2023 creamos una nueva columna con un identificador unico concatenando el día y el ticker
only_2023_df["ID"] = only_2023_df["Date"].astype(str) + " " + only_2023_df["Symbol"]

In [45]:
#Ordenamos las columnas para poner el ID lo primero
cols = list(only_2023_df)
cols = cols[-1:]+cols[:-1]
only_2023_df = only_2023_df[cols]
only_2023_df

,ID,Date,Symbol,Open,High,Low,Close,Volume,Pct_Change
0,2023-01-03 AAPL,2023-01-03,AAPL,130.280,130.9000,124.170,125.07,112117471,-3.74
1,2023-01-04 AAPL,2023-01-04,AAPL,126.890,128.6557,125.080,126.36,89113633,1.03
2,2023-01-05 AAPL,2023-01-05,AAPL,127.130,127.7700,124.760,125.02,80962708,-1.06
3,2023-01-06 AAPL,2023-01-06,AAPL,126.010,130.2900,124.890,129.62,87754715,3.68
4,2023-01-09 AAPL,2023-01-09,AAPL,130.465,133.4100,129.890,130.15,70790813,0.41
...,...,...,...,...,...,...,...,...,...
12745,2023-12-22 XOM,2023-12-22,XOM,102.310,102.9400,101.820,101.91,12927831,0.18
12746,2023-12-26 XOM,2023-12-26,XOM,102.740,103.0300,102.115,102.14,16835144,0.23
12747,2023-12-27 XOM,2023-12-27,XOM,102.040,102.5500,101.335,101.66,14535713,-0.47
12748,2023-12-28 XOM,2023-12-28,XOM,101.390,101.6100,100.130,100.19,16329252,-1.45


In [46]:
#Guardamos el df de 2023 en un CSV
only_2023_df.to_csv("Only_2023.csv", index=False)